In [1]:
import boto3
import json
import time

En la terminal ejecute:

* `bash start-sam.sh`

In [ ]:
print("Luego ejecute:\n")
print(">> sudo su")
print(">> cd /var/lib/mysql-files/")
print(">> ls -lrt")
print(">> mysql -u dbuser -pPassword@123")
print("  > use imdb;")
print("  > show tables;")

* Luego ejecute los comandos en el archivo `quer1.sql`

A título ilustrativo, a continuación se muestra un diagrama lógico que representa la relación entre varias tablas de origen que alojan el conjunto de datos IMDb.

* La tabla `title_basics` contiene las películas publicadas en EE.UU. después del año 2000. tconst es una clave alfanumérica asignada de forma exclusiva a cada película.
* `title_akas` contiene las regiones, los idiomas y los títulos de las películas publicadas. Es una relación 1:many con la tabla title_basics.
* `title_ratings` tiene la clasificación de las películas y el recuento de votos. Para este ejercicio, podemos suponer que la información se actualiza con alta frecuencia tras el estreno de la película. Tiene una relación 1:1 con la tabla title_basics
* `title_principals` contiene información sobre el reparto y el equipo. Tiene una relación 1:many con la tabla title_basics.
* `title_crew` tiene información sobre el guionista y el director. Tiene una relación 1:1 con la tabla title_basics.
* `name_basics` tiene información sobre el reparto y el equipo. Cada miembro tiene asignado un valor nconst único.
<br><br>

![Image](https://amazon-dynamodb-labs.com/images/migration31.jpg)

In [ ]:
!pip install mysql-connector-python -q

In [2]:
import os
import boto3
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
from decimal import Decimal
import json
from datetime import datetime
import time
import shutil

In [19]:
host = "3.231.154.66"
try:
    connection_config_dict = {
        'user': 'dbuser',
        'password': 'Password@123',
        'host': host,
        'database': 'imdb',
        'raise_on_warnings': True,
        'use_pure': False,
        'autocommit': True,
        'pool_size': 32
    }
    connection = mysql.connector.connect(**connection_config_dict)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        cursor = connection.cursor()
        # global connection timeout arguments
        global_connect_timeout = 'SET GLOBAL connect_timeout=655180'
        global_wait_timeout = 'SET GLOBAL connect_timeout=655180'
        global_interactive_timeout = 'SET GLOBAL connect_timeout=655180'

        cursor.execute(global_connect_timeout)
        cursor.execute(global_wait_timeout)
        cursor.execute(global_interactive_timeout)

        connection.commit()


except Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL Server version  8.0.32
Your connected to database:  ('imdb',)


In [12]:
cursor.execute("use imdb;")

In [16]:
cursor.execute("show tables;")
tables = cursor.fetchall()
print(tables)

[('name_basics',), ('title_akas',), ('title_basics',), ('title_crew',), ('title_principals',), ('title_ratings',)]


In [22]:
with open("query1.sql", "r") as query_sql:
        query = query_sql.read()
cursor.execute(query)

In [23]:
cursor.execute("select count(*) from imdb.movies;")
total_movies = cursor.fetchone()
print("Total movies: ", total_movies[0])

Total movies:  866763


In [25]:
query = "select * from imdb.movies"
result_dataFrame = pd.read_sql(query, connection)

/tmp/ipykernel_6238/3631294219.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(query, connection)


In [26]:
cursor.close()
connection.close()
print("MySQL connection is closed")

MySQL connection is closed


In [ ]:
#result_dataFrame.to_csv('movies_compress.csv.gz', compression='gzip')

#### Read dataframe

In [5]:
df = pd.read_csv("movies_compress.csv.gz")
df = df.sort_values(by=['tconst']).reset_index().drop('Unnamed: 0', axis=1)
df = df.where(pd.notnull(df), None)

In [27]:
WITH_PROFILE = False
region = 'us-east-1'
table_name = 'movies'
    
if WITH_PROFILE:
    profile = 'my-profile'
    session = boto3.Session(profile_name=profile, region_name=region)
    dynamodb = session.resource('dynamodb', region_name=region)
else:
    dynamodb = boto3.resource('dynamodb', region_name=region)

try:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'id',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'sk',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'id',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'sk',
                'AttributeType': 'S'
            },
        
        ],
        BillingMode="PAY_PER_REQUEST"
    )

    # Wait until the table exists.
    table.wait_until_exists()
except Exception as e:
    print(e)

table = dynamodb.Table(table_name)
print(table)

dynamodb.Table(name='movies')


In [7]:
def generate_json(item):
    n_errors = []
    js = {
        "id": item["tconst"],
        "sk": item["nconst"],
    }
    if item["ordering"]: 
        try:
            js["ordering"] = int(item["ordering"])
        except:
            n_errors.append(item)
    if item["category"]: 
        js["category"] = item["category"]
    if item["genres"]: 
        js["genres"] = item["genres"].split(",")
    if item["job"]: 
        js["job"] = item["job"]    
    if item["characters"]:
        try:
            js["characters"] = json.loads(item["characters"])
        except:
            n_errors.append(item)
    if item["primaryName"]:
        js["primaryName"] = item["primaryName"]
    if item["writers"]: 
        js["writers"] = item["writers"].split(",")
    return js, n_errors

In [8]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    
    if hour > 1:
        return "{} hours, {} minutes and {} seconds remaining".format(hour, minutes, seconds)
    elif hour == 1:
        return "1 hour,{} minutes and {} seconds remaining".format(minutes, seconds)
    elif hour == 0 and minutes > 0:
        return "{} minutes and {} seconds remaining".format(minutes, seconds)
    else:
        return "{} seconds remaining".format(seconds)

### Generate json files

In [2]:
js_dir = "json_data/"
error_dir = f"{js_dir}/errors/"
process_dir = f"{js_dir}/process/"

In [9]:
data = []
n_count = 0
n_end = 4000

if len(df) % n_end == 0:
    n_pass = int(len(df) / n_end)
else:
    n_pass = int(len(df) / n_end) + 1
n_breake = False

if os.path.isdir(js_dir):
    actual_files = [f for f in os.listdir(js_dir) if os.path.isfile(os.path.join(js_dir,f))]
    if len(actual_files) > 0:
        raise Exception('Folder contain files.')  
else:
    os.mkdir(js_dir)
    actual_files = []
    
for p in range(1, n_pass+1):
    print(f"Pass {p} from {n_pass}")
    if (n_count + n_end) > len(df):
        n_c = n_count
        n_end = abs(n_c - len(df))
        print(n_count + n_end)
    for i in range(n_end):
        js, errors = generate_json(df.iloc[i+n_count])
        data.append(js)
    file_json = js_dir + "/data-" + str(p) + ".json"
    with open(file_json, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
       
    data = []
    n_count += n_end
    
print("Found {} errors.".format(len(errors)))

Pass 1 from 5
Pass 2 from 5
Pass 3 from 5
Pass 4 from 5
Pass 5 from 5
16555
Found 0 errors.


### Process json files

In [15]:
COUNT_SESSION = 5
all_files = [f for f in os.listdir(js_dir) if os.path.isfile(os.path.join(js_dir,f))]
process_files = []
errors = []
count_files_end = len(all_files)

if COUNT_SESSION > len(all_files) or COUNT_SESSION < 0:
    COUNT_SESSION = len(all_files)
    int_count_session = COUNT_SESSION
elif COUNT_SESSION == 0:
    raise Exception('COUNT_SESSION cannot be 0')
else:
    int_count_session = COUNT_SESSION

for file in all_files:
    if COUNT_SESSION < 0:
        print(f"Process: {file}  -  Remaing total: {count_files_end} files.")
    else:
        rest_cs = abs(int_count_session - COUNT_SESSION) + 1
        print(f"Process: {file}  -  Remaing total: {count_files_end} files  -  {rest_cs}/{int_count_session} session process files.")
    
    start_time = time.time()
    
    with open(js_dir + file) as json_data:
        parsed = json.load(json_data, parse_float=Decimal)
    with table.batch_writer() as writer:
        for item in parsed:
            try:
                writer.put_item(Item=item)
            except:
                print(f"[ERROR] In the item: {item}")
                errors.append(item)
    
    tot_sec = round(time.time() - start_time)
    reamin_sec = convert(tot_sec * (COUNT_SESSION - 1))
    print("-- {} --".format(reamin_sec))
    
    if not os.path.isdir(process_dir):
        os.mkdir(process_dir)
    shutil.move(js_dir + file, process_dir + file)
    
    count_files_end -= 1
    COUNT_SESSION -= 1
    if COUNT_SESSION == 0:
        break
   
if len(errors) != 0:
    print("Found {} error/s.".format(len(errors)))
    if not os.path.isdir(error_dir):
        os.mkdir(error_dir)
    json_object = json.dumps(errors, indent=4)
    now = datetime.now()
    file_name = "errors-" + now.strftime("%d-%m-%Y-") + now.strftime("%H%M%S") + ".json"
    
    # Writing to sample.json
    with open(error_dir + "/" + file_name, "w") as outfile:
        outfile.write(json_object)
else:
    print("No errors were found.")

Process: data-4.json  -  Remaing total: 5 files  -  1/5 session process files.
-- 1 minutes and 48 seconds remaining --
Process: data-2.json  -  Remaing total: 4 files  -  2/5 session process files.
-- 1 minutes and 18 seconds remaining --
Process: data-3.json  -  Remaing total: 3 files  -  3/5 session process files.
-- 54 seconds remaining --
Process: data-5.json  -  Remaing total: 2 files  -  4/5 session process files.
-- 4 seconds remaining --
Process: data-1.json  -  Remaing total: 1 files  -  5/5 session process files.
-- 0 seconds remaining --
No errors were found.
